In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    """
    define layers, , 
    """
    BATCH_SIZE = 100
    EPOCHS = 18
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)

    def convs(self, x):
        # get flattened linear matrix shape
        x = F.max_pool2d(
            F.relu(self.conv1(x)), (2, 2)
        )
        x = F.max_pool2d(
            F.relu(self.conv2(x)), (2, 2)
        )
        x = F.max_pool2d(
            F.relu(self.conv3(x)), (2, 2)
        )

        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]

        return x

    def forward(self, x):
        # forward function
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)   

In [12]:
class CatDogClassifier():
    epochs = 9
    batch_size = 100
    img_size = 50
    dog = "PetImages/Dog"
    cat = "PetImages/Cat"
    labels = {
        cat: 0,
        dog: 1
    }
    training_data = []
    dog_count = 0
    cat_count = 0
    net = None
    device = None
    test_percentage = 0.1
    train_X = None
    train_y = None
    test_X = None
    test_y = None
    
    def __init__(self):
        self.net = Net()
        self.check_device()
        
    def check_device(self):
        """
        check if cuda is available then set device
        """
        
        if torch.cuda.is_available():
            device = torch.device("cuda:0")
            count = torch.cuda.device_count()
            print(f"Running on GPU, {count}")
        else:
            device = torch.devive("cpu")
            print("Running on GPU")
        self.device = device
        self.net.to(device)
        return self.device

    def split_training_data(self):
        """
        represent training data as X, y from training data
        """
        tr = self.load_training_data()
        if tr is None:
            print("No training data found.")
            return
        else:
            X = torch.Tensor([i[0] for i in tr]).view(-1, 50, 50)
            X = X/255.0
            y = torch.Tensor([x[1] for x in tr])
            
            test_value_size = int(len(X) * self.test_percentage)
            print(f"Test value size: {test_value_size}")
    
            self.train_X = X[:-test_value_size]
            self.train_y = y[:-test_value_size]
            
            self.test_X = X[-test_value_size:]
            self.test_y = y[-test_value_size:]

    def train_neural_net(self):
        """
        Train Net on Data
        """
        optimizer = optim.Adam(
            self.net.parameters(), lr=0.001
        )
        loss_function = nn.MSELoss()
        for epoch in range(self.epochs):
            for i in tqdm(range(0, len(self.train_X), self.batch_size)):
                batch_x = self.train_X[i:i + self.batch_size].view(-1, 1, 50, 50).to(self.device)
                batch_y = self.train_y[i:i + self.batch_size].to(self.device)
        
                self.net.zero_grad()
                outputs = self.net(batch_x)
                loss = loss_function(outputs, batch_y)
                loss.backward()
                optimizer.step()
    
            print(f"Epoch: {epoch}, Loss: {loss}")

    def test_neural_net(self):
        """
        Test Net accouracy
        """
        correct = 0
        total = 0
        with torch.no_grad():
            for i in tqdm(range(len(self.test_X))):
                real_class = torch.argmax(self.test_y[i])
                net_out = self.net(
                    self.test_X[i].view(-1, 1, 50, 50).to(self.device)
                )[0]
                predicted_class = torch.argmax(net_out)
                if predicted_class == real_class:
                    correct += 1
                total += 1
        print("Accuracy: ", round(correct/total, 3))
        
    def make_training_data(self):
        """
        open files in folder, grayscale then resize
        append to training data as array and one hot vector
        """
        print("reading files...")
        for label in self.labels:
            print(label)
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(
                        img, 
                        (self.img_size, self.img_size)
                    )
                    self.training_data.append(
                        [
                            np.array(img), 
                            np.eye(2)[self.labels[label]]
                        ]
                    )
                    if label == self.cat:
                        self.cat_count += 1
                    else:
                        self.dog_count +=1
                        
                except Exception as e:
                    pass
        
        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print("Cats: ", self.cat_count)
        print("Dogs: ", self.dog_count)

    def load_training_data(self):
        if not os.path.exists("training_data.npy"):
            print("Training data save does not exists!")
            return None
        self.training_data = np.load("training_data.npy", allow_pickle=True)
        print("Training data loaded...")
        return self.training_data
                    

In [13]:
c = CatDogClassifier()

Running on GPU, 1


In [14]:
c.split_training_data()

Training data loaded...
Test value size: 2494
Train X: 22452, Test X:2494


In [15]:
c.train_neural_net()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:28<00:00,  8.00it/s]


Epoch: 0, Loss: 0.21663996577262878


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:26<00:00,  8.59it/s]


Epoch: 1, Loss: 0.1707753688097


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:26<00:00,  8.63it/s]


...

Epoch: 6, Loss: 0.09469141066074371


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:26<00:00,  8.61it/s]


Epoch: 7, Loss: 0.10218621790409088


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:26<00:00,  8.61it/s]

Epoch: 8, Loss: 0.11587132513523102


In [16]:
c.test_neural_net()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2494/2494 [00:03<00:00, 764.54it/s]

Accuracy:  0.731


In [35]:
# testing on random images
cat = '4.jpg'
img = cv2.imread(cat, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(
    img, 
    (c.img_size, c.img_size)
)
np_img = np.array(img)
np_img

array([[153, 150, 154, ...,  25,  35,  37],
       [181, 182, 182, ...,  27,  34,  34],
       [200, 204, 208, ...,  27,  22,  42],
       ...,
       [ 18,   9,  10, ...,  55,  34,  72],
       [ 17,   9,   8, ...,  67,  45,  30],
       [ 20,  10,   9, ...,  52,  38,  41]], dtype=uint8)

In [36]:
X = torch.Tensor(np_img).view(-1, 1, 50, 50)

In [37]:
output = c.net(X.to(c.device))

In [38]:
torch.argmax(output)

tensor(1, device='cuda:0')